In [ ]:
import pandas as pd
import igraph as ig
import networkx as nx
from igraph import Graph
import csv
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np

### Read in data

In [ ]:
top_10k_addr = pd.read_csv('../data/top_10k_addr.csv', index_col=0)
top_10k_addr.head()

In [ ]:
from_addresses = []

with open('../data/from_addresses.csv', mode ='r') as file:   
       csvFile = csv.reader(file)
        
       for line in csvFile:
            from_addresses.append(line[0])

In [ ]:
to_addresses = []

with open('../data/to_addresses.csv', mode ='r') as file:   
       csvFile = csv.reader(file)
        
       for line in csvFile:
            to_addresses.append(line[0])

In [ ]:
unique_addresses = []

with open('../data/unique_addresses.csv', mode ='r') as file:   
       csvFile = csv.reader(file)
        
       for line in csvFile:
            unique_addresses.append(line[0])

In [ ]:
print(len(from_addresses))
print(len(to_addresses))
print(len(unique_addresses))

### Create dataframe

In [ ]:
# transactions = {'Source': from_addresses, 'Target': to_addresses}
# transactions_df = pd.DataFrame(data=transactions)
# transactions_df.head()

### Create graph from df

In [ ]:
# g = Graph.DataFrame(transactions_df, directed=True)

### Assign all edges weight 1 and sum parallel edges

In [ ]:
# g.es['weight'] = 1

In [ ]:
# g.simplify(combine_edges='sum')

In [ ]:
# edges = ig.EdgeSeq(g)
# weights = edges['weight']

In [ ]:
# weights[0:5]

## get weights of all incident edges to a vertex
### out weight sum equivalent to send count, in equivalent to receive count

In [ ]:
# len(g.es[g.incident('0xf60c2ea62edbfe808163751dd0d8693dcb30019c')])

# Build dataset for tx involving top 10k addr 

In [ ]:
#df with only top 10k addresses
unique_addr_cm = pd.DataFrame(data=top_10k_addr['address'])
unique_addr_cm.head()

In [ ]:
#transaction lists involving at least 1 top 10k address
from_addr_top = []
to_addr_top = []

from_addr_is_top = pd.Series(from_addresses).isin(unique_addr_cm['address'])
to_addr_is_top = pd.Series(to_addresses).isin(unique_addr_cm['address'])

for index in range(len(from_addr_is_top)):
    if from_addr_is_top[index] or to_addr_is_top[index]:
        from_addr_top.append(from_addresses[index])
        to_addr_top.append(to_addresses[index])


In [ ]:
len(from_addr_top)

In [ ]:
transactions = {'Source': from_addr_top, 'Target': to_addr_top}
transactions_df = pd.DataFrame(data=transactions)
# transactions_df.to_csv('top_addr_tx.csv')

## Sample top transactions

In [ ]:
min_tx = 1
top_dict = {}
for addr in unique_addr_cm['address']:
    top_dict[addr] = 0
print(len(top_dict.keys()))
from_addr_reduced = []
to_addr_reduced = []
for index in range(len(from_addr_top)):
    if from_addr_top[index] in top_dict.keys():
        from_addr = from_addr_top[index]
        to_addr = to_addr_top[index]
        if top_dict[from_addr] < min_tx:
            from_addr_reduced.append(from_addr)
            to_addr_reduced.append(to_addr)
            top_dict[from_addr] += 1
            if to_addr in top_dict.keys():
                top_dict[to_addr] += 1
        elif to_addr_top[index] in top_dict.keys():
            to_addr = to_addr_top[index]
            from_addr = from_addr_top[index]
            if top_dict[to_addr] < min_tx:
                from_addr_reduced.append(from_addr)
                to_addr_reduced.append(to_addr)
                top_dict[to_addr] += 1
                if from_addr in top_dict.keys():
                    top_dict[from_addr] += 1
    elif to_addr_top[index] in top_dict.keys():
        to_addr = to_addr_top[index]
        from_addr = from_addr_top[index]
        if top_dict[to_addr] < min_tx:
            from_addr_reduced.append(from_addr)
            to_addr_reduced.append(to_addr)
            top_dict[to_addr] += 1
            if from_addr in top_dict.keys():
                top_dict[from_addr] += 1
    

In [ ]:
top_dict.values()

In [ ]:
len(from_addr_reduced)

In [ ]:
transactions_reduced = {'Source': from_addr_reduced, 'Target': to_addr_reduced}
transactions_reduced_df = pd.DataFrame(data=transactions_reduced)

In [ ]:
g = Graph.DataFrame(transactions_reduced_df, directed=True)

g.es['weight'] = 1
g.simplify(combine_edges='sum')

edges = ig.EdgeSeq(g)
weights = edges['weight']

In [ ]:
transactions_nx = g.to_networkx()

## Node embedding

In [ ]:
from node2vec import Node2Vec
node2vec = Node2Vec(transactions_nx, dimensions=64, walk_length=100, num_walks=150, weight_key='weight', workers=1, temp_folder='C:/Users/David/Repos/School/DataScience/course_project/temp')


In [ ]:
model = node2vec.fit(window=10, min_count=4, batch_words=1)

In [ ]:
# model.save('embedded_nodes.txt')
# model.wv.save_word2vec_format('embedded_word2vec.txt')
# add
#index 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64


In [ ]:
embedding_vectors = pd.read_csv('../data/embedded_word2vec.txt', delimiter=' ')

In [ ]:
ev_df = pd.DataFrame(data=embedding_vectors)
ev_df = ev_df.set_index('index')
ev_df = ev_df.sort_index()
ev_df.index
ev_df.head()

In [ ]:
addresses = []
for vertex_num in ev_df.index:
    addresses.append(g.vs[vertex_num]['name'])
ev_df['address'] = addresses
ev_df.head()
ev_df_input = ev_df.drop('address', axis=1)

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(ev_df_input)
transformed_df = pca.transform(ev_df_input)

In [ ]:
x_vals = []
y_vals = []
for val in transformed_df:
    x_vals.append(val[0])
    y_vals.append(val[1])
    
plt.scatter(x_vals, y_vals)
plt.show()

In [ ]:
embed_out_df = pd.DataFrame(data=ev_df['address'])

In [ ]:
embed_out_df['embed1'] = x_vals
embed_out_df['embed2'] = y_vals

In [ ]:
embed_out_df = embed_out_df.set_index('address')

In [ ]:
embed_out_df.head()

In [ ]:
len(unique_addr_cm)

In [ ]:
unique_addr_cm.head()

In [ ]:
top_embed1 = []
top_embed2 = []
for addr in unique_addr_cm['address']:
    if addr in embed_out_df.index:
        top_embed1.append(embed_out_df.loc[addr]['embed1'])
        top_embed2.append(embed_out_df.loc[addr]['embed2'])
print(len(top_embed1))
unique_addr_cm['embed1'] = top_embed1
unique_addr_cm['embed2'] = top_embed2

In [ ]:
unique_addr_cm.head()
# unique_addr_cm.to_csv('unique_addr_embed.csv')

In [ ]:
unique_addr_cm.head()

## Graph for tx involving top 10k addr

In [ ]:
# #graph of tx involving top 10k addresses
# g = Graph.DataFrame(transactions_df, directed=True)

# g.es['weight'] = 1
# g.simplify(combine_edges='sum')

# edges = ig.EdgeSeq(g)
# weights = edges['weight']

In [ ]:
# import numpy as np
# np.log(g.es['weight'])
# print(int(min(np.log(g.es['weight']))))
# print(int(max(np.log(g.es['weight']))))
# print(g.es[0].attributes())

In [ ]:
# transactions_nx = g.to_networkx()

In [ ]:
# from node2vec import Node2Vec
# node2vec = Node2Vec(transactions_nx, dimensions=64, walk_length=10, num_walks=5, workers=1, temp_folder='C:/Users/David/Repos/School/DataScience/course_project/temp')


In [ ]:
# model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Centrality measures

### degree

In [ ]:
#corresponds to the number of unique id connections an address has
degrees = g.degree(unique_addr_cm['address'])
unique_addr_cm['degrees'] = degrees

In [ ]:
unique_addr_cm.head()

In [ ]:
in_degrees = g.indegree(unique_addr_cm['address'])
unique_addr_cm['in_degree'] = in_degrees
print(unique_addr_cm.head())

In [ ]:
out_degrees = g.outdegree(unique_addr_cm['address'])
unique_addr_cm['out_degree'] = out_degrees
print(unique_addr_cm.head())

In [ ]:
plt.hist(np.log([x+1 for x in degrees]), bins=50)
plt.title('Counts of log transformed node degrees')
plt.show()

In [ ]:
print(min(unique_addr_cm['degrees']))
print(max(unique_addr_cm['degrees']))
print(len(unique_addr_cm['degrees']))

### page rank

In [ ]:
page_rank = g.pagerank(vertices=unique_addr_cm['address'],weights=weights)

In [ ]:
len(page_rank)

In [ ]:
unique_addr_cm['page_rank'] = page_rank
unique_addr_cm.head()

In [ ]:
top_rank_addr = unique_addr_cm.sort_values('page_rank', ascending=False)
top_rank_addr.head(20)

In [ ]:
plt.hist(np.log([x+1 for x in page_rank]), bins=100)
plt.title('Counts of log transformed node page rank')
plt.show()

In [ ]:
print(min(unique_addr_cm['page_rank']))
print(max(unique_addr_cm['page_rank']))

### closeness

In [ ]:
# transactions_df_100k = transactions_df[0:100000]
# #graph of tx involving top 10k addresses
# g2 = Graph.DataFrame(transactions_df_100k, directed=True)

# g2.es['weight'] = 1
# g2.simplify(combine_edges='sum')

# edges2 = ig.EdgeSeq(g2)
# weights2 = edges2['weight']

In [ ]:
# print(g2.vs[0])

In [ ]:
close_vals = g.closeness(vertices=unique_addr_cm['address'], weights=weights)
unique_addr_cm['closeness'] = close_vals
unique_addr_cm.head()

In [ ]:
unique_addr_cm['closeness'] = unique_addr_cm['closeness'].fillna(0)

In [ ]:
import numpy as np
plt.hist(np.log([x+1 for x in close_vals]), bins=50)
plt.title('Counts of log transformed node closeness')
plt.show()

In [ ]:
unique_addr_cm.isnull().value_counts()

### betweeness

In [ ]:
between_vals = g.betweenness(vertices=unique_addr_cm['address'], weights=weights)
unique_addr_cm['betweeness'] = between_vals
unique_addr_cm.head()

In [ ]:
unique_addr_cm.head()

In [ ]:
import numpy as np
plt.hist(np.log([x+1 for x in between_vals]), bins=50)
plt.title('Counts of log transformed node betweenness')
plt.show()

# Log transformed dataframe

In [ ]:
unique_addr_cm_log = pd.DataFrame()
unique_addr_cm_log['address'] = unique_addr_cm['address']
unique_addr_cm_log['embed1'] = np.log([x+1 for x in unique_addr_cm['embed1']])
unique_addr_cm_log['embed2'] = np.log([x+1 for x in unique_addr_cm['embed2']])
unique_addr_cm_log['degree'] = np.log([x+1 for x in unique_addr_cm['degrees']])
unique_addr_cm_log['in_degree'] = np.log([x+1 for x in unique_addr_cm['in_degree']])
unique_addr_cm_log['out_degree'] = np.log([x+1 for x in unique_addr_cm['out_degree']])
unique_addr_cm_log['page_rank'] = np.log([x+1 for x in unique_addr_cm['page_rank']])
unique_addr_cm_log['closeness'] = np.log([x+1 for x in unique_addr_cm['closeness']])
unique_addr_cm_log['closeness'] = unique_addr_cm_log['closeness'].fillna(0)
unique_addr_cm_log['betweeness'] = np.log([x+1 for x in unique_addr_cm['betweeness']])
unique_addr_cm_log = unique_addr_cm_log.set_index('address')

In [ ]:
# unique_addr_cm_log.to_csv('graph_data_out_log.csv')
# unique_addr_cm.to_csv('graph_data_out.csv')

# Histograms

In [ ]:
plt.hist(unique_addr_cm_log['embed1'], bins=100)
plt.title('Node Embedding1')
plt.show()

In [ ]:
plt.hist(unique_addr_cm['embed2'], bins=100)
plt.title('Node Embedding2')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['degree'], bins=100)
plt.title('Degree')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['in_degree'], bins=100)
plt.title('In Degree')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['out_degree'], bins=100)
plt.title('Out Degree')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['page_rank'], bins=100)
plt.title('Page Rank')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['closeness'], bins=50)
plt.title('Closeness')
plt.show()

In [ ]:
plt.hist(unique_addr_cm_log['betweeness'], bins=50)
plt.title('Betweeness')
plt.show()

In [ ]:
print(unique_addr_cm_log.columns)

# Scatterplots

In [ ]:
fig, ax = plt.subplots(8, 8)
fig.set_size_inches(17, 8)
fig.tight_layout()
col = 0
for i in range(8):
    ax[0][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['embed1'])
    ax[0][i].set_title(unique_addr_cm_log.columns[col] +' vs. Embedding 1')
    col += 1
col = 0
for i in range(8):
    ax[1][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['embed2'])
    ax[1][i].set_title(unique_addr_cm_log.columns[col] +' vs.Embedding 2')
    col += 1
col = 0
for i in range(8):
    ax[2][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['degree'])
    ax[2][i].set_title(unique_addr_cm_log.columns[col] +' vs. Degree')
    col += 1
col = 0
for i in range(8):
    ax[3][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['in_degree'])
    ax[3][i].set_title(unique_addr_cm_log.columns[col] +' vs. In Degree')
    col += 1
col = 0
for i in range(8):
    ax[4][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['out_degree'])
    ax[4][i].set_title(unique_addr_cm_log.columns[col] +' vs. Out Degree')
    col += 1
col = 0 
for i in range(8):
    ax[5][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['page_rank'])
    ax[5][i].set_title(unique_addr_cm_log.columns[col] +' vs. Page Rank')
    col += 1
col = 0
for i in range(8):
    ax[6][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['closeness'])
    ax[6][i].set_title(unique_addr_cm_log.columns[col] +' vs. Closeness')
    col += 1
col = 0
for i in range(8):
    ax[7][i].scatter(unique_addr_cm_log[unique_addr_cm_log.columns[col]], unique_addr_cm_log['betweeness'])
    ax[7][i].set_title(unique_addr_cm_log.columns[col] +' vs. Betweenness')
    col += 1
plt.show()